In [213]:
import pandas as pd
import numpy as np
#from shapely.geometry import  Point
#import geopandas as gpd
import matplotlib.pyplot as plt
#import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder
#import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
#import contextily as ctx
#import geoplot as gplt
#import lightgbm as lgb
#import eli5
#from eli5.sklearn import PermutationImportance
#from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
#from pdpbox import pdp, get_dataset, info_plots
#import shap

In [214]:
ds = pd.read_csv('datasets/SanFranciscoCriminality.csv', parse_dates=['Dates'])
ds.drop(columns=['Descript'], inplace=True)
encoder = OrdinalEncoder()
ds['Category'] = encoder.fit_transform(ds[['Category']])

train_df, test_df = train_test_split(ds, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_x = train_df.drop(columns=['Category'])
train_y = train_df['Category']

val_x = val_df.drop(columns=['Category'])
val_y = val_df['Category']

test_x = test_df.drop(columns=['Category'])
test_y = test_df['Category']    

In [215]:
resolution = []
resolution.append(train_x['Resolution'].str.split(',').apply(len))
print(np.unique(np.array(resolution)))

def second_resolution(elem):
    return elem.split(',')[1] if len(elem.split(',')) > 1 else 'NONE'

def resolution(dataset):
    print(dataset.columns)
    dataset['Resolution_1'] = dataset['Resolution'].str.split(',').str[0]
    dataset['Resolution_2'] = dataset['Resolution'].map(second_resolution)
    return dataset.drop(columns=['Resolution'])
        

[1 2]


In [216]:
train_x.loc[~train_x['Y'].astype(str).str.startswith('37'), 'Y'] = np.nan
train_x.loc[~train_x['X'].astype(str).str.startswith('-122'), 'X'] = np.nan

imp = SimpleImputer(strategy='mean')

for district in train_x['PdDistrict'].unique():
    train_x.loc[train_x['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(train_x.loc[train_x['PdDistrict'] == district, ['X', 'Y']])
print(train_x['Y'].isnull().sum())
print(train_x['X'].isnull().sum())
def feature_engineering(data):
    data = resolution(data)
    data['Date'] = pd.to_datetime(data['Dates'].dt.date)
    data['n_days'] = (data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    
    data.drop(columns=['Dates','Date','Address'], inplace=True)
        
    return data

def fs(dataset, dataset_y):
    str_path = 'objects/SFCriminality.npy'
    if not os.path.exists(str_path):
        dataset['label'] = dataset_y
        correlation_matrix=dataset.corr()
        features = correlation_matrix['label'][(correlation_matrix['label']>=0.01) | (correlation_matrix['label']<=-0.01)].index
        features = features.drop('label')
        print(correlation_matrix['label'].sort_values(ascending=False))
        print(features)
        np.save(str_path, features)
    features = np.load(str_path, allow_pickle=True)
    return dataset[features]

def encoding(dataset, ohe):
    # Chiamata al preprocessing
    dataset = feature_engineering(dataset)
   
    num_cols = dataset.select_dtypes(exclude=['object', 'bool']).columns
    cat_cols = dataset.select_dtypes(include=['object', 'bool']).columns

    # Applica lo scaling ai dati numerici
    scaler = StandardScaler()
    dataset[num_cols] = scaler.fit_transform(dataset[num_cols])

    # Applica l'encoding ai dati categoriali nella lista ord_list
    encoder = OrdinalEncoder()
    dataset[cat_cols] = encoder.fit_transform(dataset[cat_cols])
    dataset = fs(dataset, train_y)

    if ohe[0] in dataset.columns:
        dataset = pd.get_dummies(dataset, columns=ohe)

    return dataset.astype(float), scaler, encoder

0
0


In [217]:
train_x, scaler, encoder = encoding(train_x, ['Block'])
print(train_x.dtypes)

Index(['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y'], dtype='object')
PdDistrict      float64
X               float64
Y               float64
Resolution_1    float64
Resolution_2    float64
n_days          float64
Year            float64
Hour            float64
Minute          float64
Block_0.0       float64
Block_1.0       float64
dtype: object
